In [ ]:
# 1. **Point with distance and repr**

import math


class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def distance_to_origin(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)

    def __repr__(self):
        return f"Point(x={self.x}, y={self.y})"


# Test
p = Point(3, 4)
assert p.distance_to_origin() == 5
assert "Point(" in repr(p)
print(repr(p))

In [ ]:
# 2. **User as dataclass with validation**

from dataclasses import dataclass, field


@dataclass
class User:
    name: str
    email: str
    domain: str = field(init=False)

    def __post_init__(self):
        # Normalize
        self.name = self.name.strip()
        self.email = self.email.strip().lower()

        # Validate email
        if self.email.count('@') != 1 or '.' not in self.email.split('@')[1]:
            raise ValueError(f"Invalid email: {self.email}")

        # Set domain property
        self.domain = self.email.split('@')[1]


# Test
u = User("  Ada Lovelace ", "ADA@EXAMPLE.COM ")
assert u.name == "Ada Lovelace"
assert u.domain == "example.com"
print(u)

In [ ]:
# 3. **Class variable: counting instances**

class Counter:
    """Keep a class-level count of created instances in Counter.created."""
    created = 0

    def __init__(self):
        Counter.created += 1


# Test
c1, c2 = Counter(), Counter()
assert Counter.created == 2
print(Counter.created)  # 2

In [ ]:
# 4. **Vector2D with rich ops (immutable)**

class Vector2D:
    def __init__(self, x, y):
        self._x = float(x)
        self._y = float(y)

    @property
    def x(self):
        return self._x

    @property
    def y(self):
        return self._y

    def __add__(self, other):
        if not isinstance(other, Vector2D):
            return NotImplemented
        return Vector2D(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        if not isinstance(other, Vector2D):
            return NotImplemented
        return Vector2D(self.x - other.x, self.y - other.y)

    def __mul__(self, scalar):
        if not isinstance(scalar, (int, float)):
            return NotImplemented
        return Vector2D(self.x * scalar, self.y * scalar)

    def __rmul__(self, scalar):
        return self.__mul__(scalar)

    def __eq__(self, other):
        if not isinstance(other, Vector2D):
            return NotImplemented
        return self.x == other.x and self.y == other.y

    def __hash__(self):
        return hash((self.x, self.y))

    def __repr__(self):
        return f"Vector2D({self.x}, {self.y})"

    # Make immutable by blocking setattr
    def __setattr__(self, name, value):
        if hasattr(self, name):
            raise AttributeError("Vector2D is immutable")
        super().__setattr__(name, value)


# Tests
v = Vector2D(1, 2) + Vector2D(3, 4)
assert v == Vector2D(4, 6)
assert 2 * Vector2D(1, 1) == Vector2D(2, 2)
s = {Vector2D(0, 0)}  # hashable

In [ ]:
# 5. **Rectangle with computed properties**

class Rectangle:
    def __init__(self, width, height):
        if width < 0 or height < 0:
            raise ValueError("Width and height must be non-negative")
        self._width = width
        self._height = height

    @property
    def width(self):
        return self._width

    @property
    def height(self):
        return self._height

    @property
    def area(self):
        return self.width * self.height

    @property
    def perimeter(self):
        return 2 * (self.width + self.height)


# Test
r = Rectangle(3, 4)
assert r.area == 12 and r.perimeter == 14

In [ ]:
# 6. **Abstract Shape hierarchy**

from abc import ABC, abstractmethod
import math


class Shape(ABC):
    @abstractmethod
    def area(self) -> float:
        pass


class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

    def area(self) -> float:
        return math.pi * self.radius ** 2


class Rect(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self) -> float:
        return self.width * self.height


# Tests
assert round(Circle(1).area(), 5) == round(math.pi, 5)
assert Rect(2, 3).area() == 6

In [ ]:
# 7. **Composition: Order with LineItems**

class LineItem:
    def __init__(self, name: str, qty: int, unit_price: float):
        if qty <= 0:
            raise ValueError("qty must be > 0")
        if unit_price < 0:
            raise ValueError("unit_price must be >= 0")
        self.name = name
        self.qty = qty
        self.unit_price = unit_price

    def total(self) -> float:
        return self.qty * self.unit_price


class Order:
    def __init__(self, items):
        self.items = items

    def total(self) -> float:
        return sum(item.total() for item in self.items)


# Test
o = Order([LineItem("apple", 2, 1.5), LineItem("pear", 1, 2.0)])
assert o.total() == 5.0

In [ ]:
# 8. **Context manager class**

import time


class Timer:
    def __enter__(self):
        self.start = time.perf_counter()
        return self  # return self so 'as t' works

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.perf_counter()
        self.elapsed = self.end - self.start


# Test
with Timer() as t:
    for _ in range(10000):
        pass
assert isinstance(t.elapsed, float)
print(f"Elapsed time: {t.elapsed:.6f} seconds")

In [ ]:
# 9. **Iterable/iterator protocol**

class RangeInclusive:
    def __init__(self, start, end):
        self.start = start
        self.end = end

    def __iter__(self):
        self.current = self.start
        return self

    def __next__(self):
        if self.current > self.end:
            raise StopIteration
        val = self.current
        self.current += 1
        return val


assert list(RangeInclusive(3, 5)) == [3, 4, 5]

In [ ]:
# 10. **Typing with Protocol**

from typing import Protocol, Iterable


class Closable(Protocol):
    def close(self) -> None:
        ...


def close_all(resources: Iterable[Closable]) -> int:
    count = 0
    for r in resources:
        r.close()
        count += 1
    return count


class R:
    def __init__(self):
        self.closed = False

    def close(self):
        self.closed = True


r = R()
assert close_all([r]) == 1 and r.closed is True

In [ ]:
# 11. **LRU Cache (class)**

from collections import OrderedDict


class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = OrderedDict()

    def get(self, key, default=None):
        if key not in self.cache:
            return default
        # Move key to the end to mark it as recently used
        self.cache.move_to_end(key)
        return self.cache[key]

    def set(self, key, value):
        if key in self.cache:
            # Update existing key and move to end
            self.cache.move_to_end(key)
        self.cache[key] = value
        # Evict least recently used if over capacity
        if len(self.cache) > self.capacity:
            self.cache.popitem(last=False)

    def __contains__(self, key):
        return key in self.cache


# Test
c = LRUCache(2)
c.set("a", 1)
c.set("b", 2)
_ = c.get("a")
c.set("c", 3)
assert "b" not in c and "a" in c and "c" in c

In [ ]:
# 12. **Strategy pattern: discounts**

from abc import ABC, abstractmethod


class Discount(ABC):
    @abstractmethod
    def apply(self, amount: float) -> float:
        pass


class NoDiscount(Discount):
    def apply(self, amount: float) -> float:
        return amount


class Percent(Discount):
    def __init__(self, rate: float):
        self.rate = rate  # e.g., 0.1 means 10% off

    def apply(self, amount: float) -> float:
        return amount * (1 - self.rate)


class Flat(Discount):
    def __init__(self, amount_off: float):
        self.amount_off = amount_off

    def apply(self, amount: float) -> float:
        return max(0, amount - self.amount_off)


def total_with_discount(amount, strategy: Discount) -> float:
    return strategy.apply(amount)


# Tests
assert total_with_discount(100, Percent(0.1)) == 90
assert total_with_discount(50, Flat(5)) == 45
assert total_with_discount(40, NoDiscount()) == 40